In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pprint
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
exchange = 'bybit'
user = 'your_user_name'
settings = load_settings(exchange, user)
s = 'BTCUSD'
n_days = 7

In [ ]:
#load cache if any
#agg_trades = pd.read_csv('btcusdt_agg_trades_bybit_28_days_2021-01-10.csv').set_index('trade_id')
#adf = agg_trades

In [ ]:
#otherwise
#agg_trades = await load_trades(exchange, user, s, n_days)
#adf = agg_trades

In [ ]:
# look at jackrabbit results, if any
'''
rdf = pd.concat([pd.read_csv('jackrabbit_results_grid/2021-01-10T02:47:26_inverse.csv')])
#column_names = ['grid_step', 'markups']
column_names = ['default_qty', 'grid_step', 'markups']
#column_names = ['default_qty', 'grid_step', 'margin_limit', 'markups', 'n_close_orders']

rdf.columns = column_names + list(rdf.columns[len(column_names):])
rdfs = rdf.sort_values('gain', ascending=[False])
rdfs = rdfs[~rdfs.gain.duplicated()]
rdfs.loc[:, 'pnl_over_max_margin_cost'] = rdfs.pnl_sum / rdfs.max_margin_cost
rdfs.head(30)
'''

In [ ]:
pprint.pprint(dict(rdfs.iloc[0]))

In [ ]:
# plotting method

def plot_tdf_(df_, tdf_, side_: int = 0, liq_thr=0.1):
    df_.loc[tdf_.index[0]:tdf_.index[-1]].price.plot(style='y-')
    if side_ >= 0:
        longs = tdf_[tdf_.side == 'long']
        le = longs[longs.type == 'entry']
        lc = longs[longs.type == 'close']
        ll = longs[longs.type == 'liq']
        le.price.plot(style='b.')
        longs.pos_price.plot(style='b--')
        if 'close_price' in longs.columns:
            longs.close_price.plot(style='r--')
        if 'liq_price' in longs.columns:
            longs.liq_price.where((longs.price - longs.liq_price).abs() / longs.price < liq_thr, np.nan).plot(style='k--')            
        lc.price.plot(style='ro')
        ll.price.plot(style='gx')
    if side_ <= 0:
        shrts = tdf_[tdf_.side == 'shrt']
        se = shrts[shrts.type == 'entry']
        sc = shrts[shrts.type == 'close']
        sl = shrts[shrts.type == 'liq']
        se.price.plot(style='r.')
        shrts.pos_price.plot(style='r--')
        if 'close_price' in shrts.columns:
            shrts.close_price.plot(style='b--')
        if 'liq_price' in shrts.columns:
            shrts.liq_price.where((shrts.price - shrts.liq_price).abs() / shrts.price < liq_thr, np.nan).plot(style='k--')
            
        sc.price.plot(style='bo')
        sl.price.plot(style='gx')

In [ ]:
# crop trades if desired
'''
age_limit = (time() - 60 * 60 * 24 * 5) * 1000
adf = adf[adf.timestamp > age_limit]
n_days = (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
n_days
'''

In [ ]:
# prep df for backtesting
df = prep_df(adf)
print(len(df), len(df) / len(adf))
df.price.iloc[::100].plot()

In [ ]:
settings

In [ ]:
# bybit
settings['break_on_loss'] = False
settings['default_qty'] = 1.0
settings['grid_coefficient'] = 80.0
settings['grid_spacing'] = 0.002
settings['grid_step'] = 25
settings['inverse'] = True
settings['leverage'] = 100
settings['maker_fee'] = -0.00025
settings['margin_limit'] = 0.00127
settings['markups'] = (0.0002, 0.0159)
settings['min_qty'] = 1.0
settings['n_close_orders'] = 20
settings['n_entry_orders'] = 10
settings['price_step'] = 0.5
settings['qty_step'] = 1.0
settings['soft_loss_margin_threshold'] = 0.0001
settings['symbol'] = 'BTCUSD'

In [ ]:
sort_dict_keys(settings)

In [ ]:
start_time = time()
#trades = backtest_dynamic_grid(df, settings)
trades = backtest(df, settings)
print(f'seconds elapsed {time() - start_time:.2f}')

In [ ]:
# analyze results
tdf = pd.DataFrame(trades).set_index('trade_id')
longs = tdf[tdf.side == 'long']
shrts = tdf[tdf.side == 'shrt']
le = longs[longs.type == 'entry']
lc = longs[longs.type == 'close']
se = shrts[shrts.type == 'entry']
sc = shrts[shrts.type == 'close']

margin_max = (tdf.pos_size.abs() / tdf.pos_price).max() / settings['leverage']
biggest_pos_size = tdf.pos_size.abs().max()
pnl_sum = tdf.pnl.sum()
gain = (settings['margin_limit'] + pnl_sum) / settings['margin_limit']
closest_liq = ((tdf.price - tdf.liq_price).abs() / tdf.price).min()
n_days = (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
average_daily_gain = gain ** (1 / n_days)
closes = tdf[tdf.type == 'close']
print('margin_max', margin_max)
print('biggest qty', tdf.qty.abs().max())
print('pnl_sum', pnl_sum)
print('gain', gain)
print('n_days', n_days)
print('average_daily_gain', average_daily_gain)
print('n trades', len(tdf))
print('n closes', len(closes))
print('biggest_pos_size', biggest_pos_size)
print('closest liq', closest_liq)

In [ ]:
plot_tdf_(df, tdf, liq_thr=0.1)

In [ ]:
# visualize behavior
step = 60
i = -step

In [ ]:
i += step
tdfc = tdf.iloc[i:i+step]
plot_tdf_(df, tdf.iloc[i:i+step], liq_thr=0.1)

In [ ]:
tdfc

In [ ]:
tdf.pnl.cumsum().plot()

In [ ]:
tdf.pos_size.plot()

In [ ]:
# inspect biggest trades
qty_abs = tdf.qty.abs().sort_values(ascending=False)
qty_abs.head(10)

In [ ]:
i = 0
iloc_ = tdf.index.get_loc(qty_abs.index[i])
iminus = 400
iplus = 20
tdfc = tdf.iloc[max(0, iloc_-iminus):min(iloc_+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc)

In [ ]:
ik = -1
stepp = 60

In [ ]:
ik += 1
tdfc.iloc[stepp * ik:stepp * (ik + 1)]